In [4]:
from scraper.scraper import Scraper 
bot = Scraper()
bot.accept_cookies()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/FKhan/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


In [3]:
bot.keys_search('nike caps')

In [188]:
bot.next_button()

No next button found


In [332]:
container = bot.product_container()

In [333]:
from lib2to3.pgen2 import driver
from time import time
from typing_extensions import Self
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager


In [335]:
list_properties = container.find_elements(By.XPATH, './ul/li')

In [241]:
link_list = []
for property in list_properties:
    link_list.append(property.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [242]:
import time
import uuid
property_dict = {'link':[],
                 'price':[],
                 'name':[],
                 'id' :[],
                 'UUID':[],
                }


for link in link_list[0:27]:
    bot.driver.get(link)
    time.sleep(1)
    property_dict['link'].append(link)
    price = bot.driver.find_element(By.XPATH, '//span[@class="pri"]')
    property_dict['price'].append(price.text)
    name = bot.driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
    property_dict['name'].append(name.text)
    id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
    property_dict['id'].append(id.text)
    property_dict['UUID'].append(uuid.uuid4())
    

In [243]:
property_dict

{'link': ['https://www.jdsports.co.uk/product/grey-nike-heritage-86-swoosh-cap/16474497/',
  'https://www.jdsports.co.uk/product/green-nike-sportswear-heritage86-cap/16453825/',
  'https://www.jdsports.co.uk/product/black-nike-heritage86-cap-womens/16154660/',
  'https://www.jdsports.co.uk/product/black-nike-l91-futura-cap/16069344/',
  'https://www.jdsports.co.uk/product/white-nike-heritage86-cap-womens/16253133/',
  'https://www.jdsports.co.uk/product/pink-nike-featherlight-running-cap/16232870/',
  'https://www.jdsports.co.uk/product/black-nike-futura-trucker-cap/16070397/',
  'https://www.jdsports.co.uk/product/black-nike-heritage-86-cap/1250112/',
  'https://www.jdsports.co.uk/product/blue-nike-heritage-86-cap/023018/',
  'https://www.jdsports.co.uk/product/black-nike-sportswear-air-max-legacy-91-cap/16187362/',
  'https://www.jdsports.co.uk/product/black-nike-aerobill-tailwind-cap/15988034/',
  'https://www.jdsports.co.uk/product/white-nike-aerobill-tailwind-cap/16155854/',
  'ht

In [217]:
import json
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return obj.hex
        return json.JSONEncoder.default(self, obj)
j = json.dumps(property_dict, cls=UUIDEncoder, indent=4)
f = open('data.json', 'w')
print(j, file=f)
f.close()

In [244]:
import pandas as pd 
pd.DataFrame(property_dict)

,link,price,name,id,UUID
0,https://www.jdsports.co.uk/product/grey-nike-h...,£15.00,Nike Heritage '86 Swoosh Cap,Product Code: 16474497/474027,66539ded-6ed7-41c1-ae1c-5e5a1987de8c
1,https://www.jdsports.co.uk/product/green-nike-...,£25.00,Nike Sportswear Heritage86 Cap,Product Code: 16453825/474048,ea098c69-d957-47bb-9696-6338bc3d47a0
2,https://www.jdsports.co.uk/product/black-nike-...,£23.00,Nike Heritage86 Cap Women's,Product Code: 16154660/506521,0b47284d-de92-4533-9eef-e195e0720821
3,https://www.jdsports.co.uk/product/black-nike-...,£23.00,Nike L91 Futura Cap,Product Code: 16069344/393535,3d55860b-a91f-4de4-b36b-ec6b68f68408
4,https://www.jdsports.co.uk/product/white-nike-...,£23.00,Nike Heritage86 Cap Women's,Product Code: 16253133/506525,1a9fbf67-5e1e-418e-9723-d845d1a20e4c
5,https://www.jdsports.co.uk/product/pink-nike-f...,£18.00,Nike Featherlight Running Cap,Product Code: 16232870/473947,a15b7fb3-5186-4d0e-9457-8a9cef3be314
6,https://www.jdsports.co.uk/product/black-nike-...,£25.00,Nike Futura Trucker Cap,Product Code: 16070397/393536,76d2ec08-c106-4506-b8d3-f8e61d4b4184
7,https://www.jdsports.co.uk/product/black-nike-...,£17.00,Nike Heritage 86 Cap,Product Code: 1250112/012408,876d385a-6463-4408-8951-f3733f3655d7
8,https://www.jdsports.co.uk/product/blue-nike-h...,£17.00,Nike Heritage 86 Cap,Product Code: 023018/023018,9ab649f0-c1cd-4d33-954c-eb64484cf173
9,https://www.jdsports.co.uk/product/black-nike-...,£23.00,Nike Sportswear Air Max Legacy 91 Cap,Product Code: 16187362/458460,278d1caf-8a1d-43d6-b8c0-663003bd1ed0


In [360]:
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [361]:
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))

In [396]:
import urllib.request
def search_image():
        url = 'https://i8.amplience.net/t/jpl/jd_product_list?plu=jd_355655_plc&qlt=92&w=363&h=463&v=1&fmt=auto'
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(
        url, f'Nike_Hat_27.png')
        print(f'downloaded!')
try:
    search_image()
except: print('Thats not a valid url')

downloaded!
